# Api Gateway with Cognito Authentication

This example deploys a microservice with an API Gateway which uses an Amazon Cognito user pool as an authoriser. The POST method on the API Gateway allows authenticated requests only. 

1.	The first script is the json formatter, it is used to render json in a readable format.


2.	The second script creates a new Cognito user, make sure you set the variable and confirm the user. This can be done using the Cognito console.


Username: ```your name```

Email: ```your email```

Password: 
```
Mininum length: 8 
Require numbers: true
Require special character: true
Require uppercase letters: true
Require lowercase letters: true
```

3.	The login in script authenticates the new user and cognito will return a json web token (jwt).


4.	A new order is sent to the POST method endpoint along with the jwt IdToken, the POST method is configured for authenticated requests only. The API Gateway will validate the IdToken against the Cognito user pool that generated it, this is done using an 'Authorizer'.


5.	The final script calls an unauthenticated GET method to retrieve the item.


![architecture](../images/architecture_1.png "Architecture")

### json formatter 
Run the following script to create a class which will be used to render json objects in a readable format.

In [ ]:
import json, uuid
from IPython.display import display_javascript, display_html, display

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict) or isinstance(json_data, list):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;font: 12px/18px monospace !important;"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
            renderjson.set_show_to_level(2);
            document.getElementById('%s').appendChild(renderjson(%s))
        });
      """ % (self.uuid, self.json_str), raw=True)

 ### Create User
 Create a user by using the **sign_up** method on the Cognito SDK
 
 **Set the following variables:**
 
 * clientId (Copy your App client id from Cognito User Pools console)
 * Username
 * Password
 * Email
 
 
 **note:** the user will need to be confirmed either by email or through the cognito console.

In [ ]:
import json, boto3, requests, datetime

# Client ID goes here
clientId = '...'
username = '...'
password = '...'
email = '...'

client = boto3.client('cognito-idp')

signup_response = client.sign_up(
    ClientId=clientId,
    Username=username,
    Password=password,
    UserAttributes=[{'Name':'email','Value':email}])

RenderJSON(signup_response)

### Login
Login into by calling the Cognito SDK **initiate_auth** method. Amazon Cognito will return a json web token. The IdToken will be used for authentication against the API Gateway.

**note:** Make sure you **confirm the user**.

In [ ]:
auth_response = client.initiate_auth(
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={
        'USERNAME' : username,
        'PASSWORD' : password
    },
    ClientId=clientId
)

print(auth_response['AuthenticationResult'].keys())
print(auth_response['AuthenticationResult']['IdToken'])

### Call API with authentication header and token

The IdToken is passed to the API gateway as a HTTP Authorization header, the gateway validates the token using the Cognito authorizer attached to the POST method.

**Note:** Make sure you set **gwid** to your gateway id using - *gwid = '...'*

In [ ]:
#Set gateway id
gwid = '...'

#Set your AWS region, e.g. ap-southeast-2
region = '...' 

import boto3, requests, datetime
from random import randrange

url = (f'https://{gwid}.execute-api.{region}.amazonaws.com/prod/order')
headers = {'Authorization': auth_response['AuthenticationResult']['IdToken']}

city_list=["Adelaide","Brisbane","Canberra","Darwin","Geelong","Gold Coast","Hobart","Melbourne","Perth","Sydney","Wollongong"]
coffeetype_list=["Short Black","Flat White","Latte","Long Black"]
coffeesize_list=[{"size":"Small","price":"3.5"},{"size":"Medium","price":"4.0"},{"size":"Large","price":"4.5"},{"size":"x-Large","price":"5.0"}]

for i in range(10):
    accountid = 'a' + str(i)
    vendorid = 'v' + str(i)
    orderdate = str(datetime.datetime.now())
    
    #Original value-----------
    coffeesize = randrange(4)
    #-------------------------

    quantity = randrange(10)+1
    city = city_list[randrange(11)]
    eventtype="new_order"
    response = requests.post(url,json={'order':{
            'accountid': accountid,
            'orderdate':orderdate,
            'vendorid': vendorid,
            'city':city,
            'details':{
                'coffeetype': coffeetype_list[randrange(4)],
                'coffeesize': coffeesize_list[coffeesize]["size"],
                'unitprice': coffeesize_list[coffeesize]["price"],
                'quantity': quantity
            },
            'eventtype':[eventtype]
        }
    },headers=headers)

print(response)

Get the item from Amazon DynamoDB using the primary key (accountid & vendorid)

In [ ]:
response_get = requests.get(url, params={'accountid':'a0','vendorid':'v0'})

RenderJSON(response_get.json())